In [1]:
pass

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1605607978346_0004,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# %load_ext autoreload

# %autoreload 2

# %matplotlib inline

In [3]:
import datetime

import numpy as np
import pandas as pd

import dask.dataframe as dd

# import matplotlib.pyplot as plt

# from tqdm import tqdm
# tqdm.pandas() # use progress_apply instead of apply

from dask.diagnostics import ProgressBar
ProgressBar().register()


In [4]:
import logging

import os
import sys
import json
# from pyspark.sql import SparkSession

from datetime import datetime

import boto3

from s3fs import S3FileSystem
fs = S3FileSystem()

from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col

In [7]:
class PrepUIMatrix:
    def __init__(self):
        pass
    def func(self):
        pass

self = PrepUIMatrix()

In [8]:
self.path_cookie_cookie_staging = 's3://zeta-dc-ml/ai-audiences/datacloud_cookie_cookie_relations/cookie_cookie_staging/dt=2020-11-16'



In [9]:
self.path_UI_interactions_sizmek_ = 's3://zeta-dc-ml/ai-audiences/modeling/dt=2020-10-14/z31_user_item_interactions_sizmek'
self.path_UI_interactions_count_sizmek_ = 's3://zeta-dc-ml/ai-audiences/modeling/dt=2020-10-14/z41_user_item_interactions_count_sizmek'
self.path_I_count_count_sizmek_ = 's3://zeta-dc-ml/ai-audiences/modeling/dt=2020-10-14/z51_item_count_count_sizmek'

self.path_UI_interactions_zync_ = 's3://zeta-dc-ml/ai-audiences/modeling/dt=2020-10-14/z32_user_item_interactions_zync'
self.path_UI_interactions_count_zync_ = 's3://zeta-dc-ml/ai-audiences/modeling/dt=2020-10-14/z42_user_item_interactions_count_zync'
self.path_I_count_count_zync_ = 's3://zeta-dc-ml/ai-audiences/modeling/dt=2020-10-14/z52_item_count_count_zync'

self.path_UI_interactions_disqus_ = 's3://zeta-dc-ml/ai-audiences/modeling/dt=2020-10-14/z33_user_item_interactions_disqus'
self.path_UI_interactions_count_disqus_ = 's3://zeta-dc-ml/ai-audiences/modeling/dt=2020-10-14/z43_user_item_interactions_count_disqus'
self.path_I_count_count_disqus_ = 's3://zeta-dc-ml/ai-audiences/modeling/dt=2020-10-14/z53_item_count_count_disqus'



In [10]:
df = spark.read.option("inferSchema", True).parquet(self.path_cookie_cookie_staging + '/*parquet')
df.printSchema()
df.show(n=4)


root
 |-- source: string (nullable = true)
 |-- cookie_source: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- cookie_destination: string (nullable = true)
 |-- last_updated: timestamp (nullable = true)

+--------+--------------------+-----------+--------------------+-------------------+
|  source|       cookie_source|destination|  cookie_destination|       last_updated|
+--------+--------------------+-----------+--------------------+-------------------+
|33across|/yUFtV9N+3v/i2F4A...|     sizmek| 2810035062355267748|2020-09-15 00:00:00|
|33across|/yUFtV9N+rL/i2F4A...|       zync|db9cd184-149a-4f6...|2020-09-09 20:00:00|
|33across|/yUFtV9N2tL/i2F4A...|       zync|ff74ac59-285a-4b0...|2020-09-07 21:00:00|
|33across|/yUFtV9N9H7/i2F4A...|       zync|b1c47d46-ee23-46b...|2020-09-07 06:00:00|
+--------+--------------------+-----------+--------------------+-------------------+
only showing top 4 rows

In [13]:
df.filter(df.source == 'sizmek').count()

20900010516

In [12]:
df.filter(df.source == 'zync').count()

27366652183

In [11]:
df.filter(df.source == 'disqus').count()

12755004399

In [14]:
df.filter(df.destination == 'disqus').count()

12126421925

In [15]:
df.filter(df.destination == 'zync').count()

25353426842

In [16]:
df.filter(df.destination == 'sizmek').count()

23146752456

In [17]:
df.filter((df.source == 'disqus') & (df.destination == 'sizmek')).count()

3372866566

In [18]:
df.filter((df.source == 'sizmek') & (df.destination == 'disqus')).count()

2667247165

In [19]:
df.filter((df.source == 'zync') & (df.destination == 'sizmek')).count()

12084795314

In [20]:
df.filter((df.source == 'sizmek') & (df.destination == 'zync')).count()

10780176468

In [21]:
df.filter((df.source == 'disqus') & (df.destination == 'zync')).count()

3191754906

In [22]:
df.filter((df.source == 'zync') & (df.destination == 'disqus')).count()

3397344784

In [115]:
df.printSchema()

root
 |-- source: string (nullable = true)
 |-- cookie_source: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- cookie_destination: string (nullable = true)
 |-- last_updated: timestamp (nullable = true)

In [116]:
df_zync_to_sizmek_1 = df.filter((df.source == 'zync') & (df.destination == 'sizmek'))

df_zync_to_sizmek_1 = df_zync_to_sizmek_1.selectExpr('cookie_source as cookie_zync',
                                                     'cookie_destination as cookie_sizmek')

df_zync_to_sizmek_1.printSchema()


root
 |-- cookie_zync: string (nullable = true)
 |-- cookie_sizmek: string (nullable = true)

In [117]:
df_zync_to_sizmek_2 = df.filter((df.source == 'sizmek') & (df.destination == 'zync'))

df_zync_to_sizmek_2 = df_zync_to_sizmek_2.selectExpr('cookie_source as cookie_sizmek',
                                                     'cookie_destination as cookie_zync'
                                                     )
df_zync_to_sizmek_2.printSchema()

root
 |-- cookie_sizmek: string (nullable = true)
 |-- cookie_zync: string (nullable = true)

In [118]:
df_zync_to_sizmek = df_zync_to_sizmek_1.union(df_zync_to_sizmek_2)
df_zync_to_sizmek.printSchema()

root
 |-- cookie_zync: string (nullable = true)
 |-- cookie_sizmek: string (nullable = true)

In [119]:
df_zync_to_sizmek = df_zync_to_sizmek.drop_duplicates()
# df_zync_to_sizmek.count()

# df_zync_to_sizmek = (df_zync_to_sizmek
#                        .groupBy('cookie_zync')
#                        .agg(F.collect_set('cookie_dest').alias('cookie_sizmek'))
#                       )




In [120]:
df_zync_to_sizmek.printSchema()

root
 |-- cookie_zync: string (nullable = true)
 |-- cookie_sizmek: string (nullable = true)

In [121]:
df_temp = df_zync_to_sizmek.groupBy("cookie_sizmek").count().orderBy(col("count").desc())

In [122]:
df_temp.printSchema()

root
 |-- cookie_sizmek: string (nullable = true)
 |-- count: long (nullable = false)

In [123]:
df_temp = df_temp.withColumnRenamed('count','num_cookies')

In [124]:
df_temp.printSchema()

root
 |-- cookie_sizmek: string (nullable = true)
 |-- num_cookies: long (nullable = false)

In [125]:
df_temp = df_temp.groupBy("num_cookies").count().orderBy(col("num_cookies").desc())

In [126]:
df_temp.printSchema()

root
 |-- num_cookies: long (nullable = false)
 |-- count: long (nullable = false)

In [127]:
pandas_df = df_temp.toPandas()

In [128]:
pandas_df.head()

   num_cookies  count
0         5113      1
1         4168      1
2         4075      1
3         1525      1
4         1349      1

In [129]:
path = 's3://zeta-dc-ml/ai-audiences/modeling/dt=2020-10-14/z102_sizmek_zync_count.csv'
pandas_df.to_csv(path, header=True, index=False)

In [10]:
self.get_list_dates_that_can_be_processed()

number of buckets in s3://zeta-dcp-prod-private-tables/datacloud_cookie_cookies_relations_links_export/ are 132
datacloud_cookie_cookies_relations_links_export/dt=0-sample-2020-06-19/
date partition that cannot be processed: 0-sample-2020-06-19
date partition that cannot be processed: 2020-06-02-sample
date partition that cannot be processed: backfill-scylla-till-09-17
date partition that cannot be processed: bkp_2020-10-08
date partition that cannot be processed: final-test-2020-11-11
date partition that cannot be processed: sample-backfill-scylla-till-09-17
date partition that cannot be processed: test-2020-06-28
number of buckets that cannot be processed in s3://zeta-dcp-prod-private-tables/datacloud_cookie_cookies_relations_links_export/ are 7
number of buckets that can be processed in s3://zeta-dcp-prod-private-tables/datacloud_cookie_cookies_relations_links_export/ are 125

In [10]:
self.prep_cookie_cookie_map_table()

number of buckets in s3://zeta-dcp-prod-private-tables/datacloud_cookie_cookies_relations_links_export/ are 133
datacloud_cookie_cookies_relations_links_export/dt=0-sample-2020-06-19/
date partition that cannot be processed: 0-sample-2020-06-19
date partition that cannot be processed: 2020-06-02-sample
date partition that cannot be processed: backfill-scylla-till-09-17
date partition that cannot be processed: bkp_2020-10-08
date partition that cannot be processed: final-test-2020-11-11
date partition that cannot be processed: sample-backfill-scylla-till-09-17
date partition that cannot be processed: test-2020-06-28
number of buckets that cannot be processed in s3://zeta-dcp-prod-private-tables/datacloud_cookie_cookies_relations_links_export/ are 7
number of buckets that can be processed in s3://zeta-dcp-prod-private-tables/datacloud_cookie_cookies_relations_links_export/ are 126
Latest Date to process is 2020-11-16
Source read successful for date: 2020-11-16
processed cookie_cookie_map

In [6]:
import logging

import os
import sys
import json
from pyspark.sql import SparkSession

import boto3
from datetime import datetime
from urllib.parse import urlparse

# https://stackoverflow.com/questions/39235704/split-spark-dataframe-string-column-into-multiple-columns
from pyspark.sql.functions import split
from pyspark.sql.functions import collect_set, collect_list # collect_set eliminates duplicates, collect_list does not

In [7]:
PATH_ROOT = 's3://zeta-dc-ml/ai-audiences/'
PATH_COOKIE_COOKIES_RELATIONS = 's3://zeta-dcp-prod-private-tables/datacloud_cookie_cookies_relations_links_export/'


class CookieMapUpdate:

    def __init__(self,
                 path_root=PATH_ROOT,
                 path_cookie_cookies_relations=PATH_COOKIE_COOKIES_RELATIONS,
                 ):

        self.path_root = path_root
        self.path_cookie_relations_dest_root = self.path_root + 'datacloud_cookie_cookie_relations/'
        self.path_cookie_cookie_staging = self.path_cookie_relations_dest_root + 'cookie_cookie_staging/'
        self.path_disqus_to_sizmek = self.path_cookie_relations_dest_root + 'disqus_to_sizmek/'
        self.path_zync_to_sizmek = self.path_cookie_relations_dest_root + 'zync_to_sizmek/'

        self.path_cookie_cookies_relations = path_cookie_cookies_relations

        self.list_of_all_dates_that_can_be_processed = None
        self.date_processed_latest = None
        self.date_processed_latest_counter = 0

    @staticmethod
    def get_list_of_all_dates_present(bucket, prefix):
        
        s3 = boto3.client("s3")
        all_objects = s3.list_objects(Bucket=bucket,
                                      Prefix=prefix,
                                      Delimiter='/')

        list_date_buckets = []
        for o in all_objects.get('CommonPrefixes'):
            list_date_buckets.append(o.get('Prefix'))

        print(f'number of buckets in s3://{bucket}/{prefix} are {len(list_date_buckets)}')
        
        print(list_date_buckets[0])

        list_dates_present = [el.split('/')[-2].split('=')[-1] for el in list_date_buckets]

        list_dates_that_can_be_processed = []

        count = 0
        for el in list_dates_present:
            try:
                list_dates_that_can_be_processed.append(datetime.strptime(el, '%Y-%m-%d'))
            except:
                count = count + 1
                print(f'date partition that cannot be processed: {el}')

        print(f'number of buckets that cannot be processed in s3://{bucket}/{prefix} are {count}')
        print(f'number of buckets that can be processed in s3://{bucket}/{prefix} are {len(list_dates_that_can_be_processed)}')

        list_dates_that_can_be_processed.sort()

        return list_dates_that_can_be_processed

    def get_list_dates_that_can_be_processed(self):

        parsed_url = urlparse(self.path_cookie_cookies_relations, allow_fragments=False)

        bucket = parsed_url.netloc
        rest_of_path = parsed_url.path.lstrip('/')

        self.list_of_all_dates_that_can_be_processed = self.get_list_of_all_dates_present(bucket=bucket,
                                                                                          prefix=rest_of_path)

    def get_date_processed_latest(self):

        self.date_processed_latest = self.list_of_all_dates_that_can_be_processed[-1 - self.date_processed_latest_counter].strftime('%Y-%m-%d')
        print(f'Latest Date to process is {self.date_processed_latest}')

    def prep_cookie_cookie_map_table(self):

        # Read from the latest, if the latest fails, read from the second to latest, else keep going back upto 10? days
        num_read_attempts = 10
        self.get_list_dates_that_can_be_processed()
        for i in range(num_read_attempts):
            try:
                self.get_date_processed_latest()
                df = (spark.read.option("inferSchema", True)
                      .orc(os.path.join(self.path_cookie_cookies_relations,
                                        f'dt={self.date_processed_latest}',
                                        '*'
                                        )
                           )
                      )
                print(f'Source read successful for date: {self.date_processed_latest}')
                break
            except:
                # TODO: Alert AI-ML team as this could mean a change in table/schema
                print(f'Source read failed for date: {self.date_processed_latest}')
                self.date_processed_latest_counter = self.date_processed_latest_counter + 1

        if self.date_processed_latest_counter == 10:
            print('Cookie - Cookie Mapping table has issues')
            return None

        split_col = split(df['cookie'], '::')
        df = df.withColumn('source', split_col.getItem(0))
        df = df.withColumn('cookie_source', split_col.getItem(1))

        split_col = split(df['relation'], '::')
        df = df.withColumn('destination', split_col.getItem(0))
        df = df.withColumn('cookie_destination', split_col.getItem(1))

        df = df.select(df['source'], df['cookie_source'], df['destination'], df['cookie_destination'], df['last_updated'])

        try:
            df.write.parquet(os.path.join(self.path_cookie_cookie_staging,
                                          f'dt={self.date_processed_latest}'
                                          )
                             )
            print(f'processed cookie_cookie_mappings with latest date processed: {self.date_processed_latest}')
        except:
            print(f'data of cookie_cookie_mappings for date {self.date_processed_latest} might already exist')

    def prep_disqus_sizmek_map_table(self):

        bucket = self.path_cookie_cookie_staging.split('/')[2]
        rest_of_path = os.path.join(*(self.path_cookie_cookie_staging.split('/')[3:]))

        temp_list_of_dates = self.get_list_of_all_dates_present(bucket=bucket,
                                                                prefix=rest_of_path)
        date_processed_latest_ = temp_list_of_dates[-1].strftime('%Y-%m-%d')

        df = (spark.read.option("inferSchema", True)
              .parquet(os.path.join(self.path_cookie_cookie_staging,
                                    f'dt={date_processed_latest_}',
                                    '*parquet')
                       )
              )

        df_disqus_to_sizmek = df.filter((df.source == 'disqus') & (df.destination == 'sizmek'))

        df_disqus_to_sizmek = df_disqus_to_sizmek.selectExpr('cookie_source as cookie_disqus',
                                                             'cookie_destination as cookie_dest')
        df_disqus_to_sizmek = df_disqus_to_sizmek.drop_duplicates()

        df_disqus_to_sizmek = (df_disqus_to_sizmek
                               .groupBy('cookie_disqus')
                               .agg(collect_set('cookie_dest').alias('cookie_sizmek'))
                               )
        try:
            df_disqus_to_sizmek.write.parquet(os.path.join(self.path_disqus_to_sizmek,
                                                           f'dt={date_processed_latest_}'
                                                           )
                                              )
            print(f'processed disqus_to_sizmek_cookie_mappings with latest date processed: {date_processed_latest_}')
        except:
            print(f'data of disqus_to_sizmek for date {date_processed_latest_} might already exist')

    def prep_zync_sizmek_map_table(self):

        bucket = self.path_cookie_cookie_staging.split('/')[2]
        rest_of_path = os.path.join(*(self.path_cookie_cookie_staging.split('/')[3:]))

        temp_list_of_dates = self.get_list_of_all_dates_present(bucket=bucket,
                                                                prefix=rest_of_path)
        date_processed_latest_ = temp_list_of_dates[-1].strftime('%Y-%m-%d')

        df = (spark.read.option("inferSchema", True)
              .parquet(os.path.join(self.path_cookie_cookie_staging,
                                    f'dt={date_processed_latest_}',
                                    '*.parquet')
                       )
              )

        df_zync_to_sizmek = df.filter((df.source == 'zync') & (df.destination == 'sizmek'))

        df_zync_to_sizmek = df_zync_to_sizmek.selectExpr('cookie_source as cookie_zync',
                                                         'cookie_destination as cookie_dest')
        df_zync_to_sizmek = df_zync_to_sizmek.drop_duplicates()

        df_zync_to_sizmek = (df_zync_to_sizmek
                             .groupBy('cookie_zync')
                             .agg(collect_set('cookie_dest').alias('cookie_sizmek'))
                             )

        try:
            df_zync_to_sizmek.write.parquet(os.path.join(self.path_zync_to_sizmek,
                                                         f'dt={date_processed_latest_}'
                                                         )
                                            )
            print(f'processed zync_to_sizmek_cookie_mappings with latest date processed: {date_processed_latest_}')
        except:
            print(f'data of zync_to_sizmek for date {date_processed_latest_} might already exist')

    def run_step(self):
        try:
            self.prep_cookie_cookie_map_table()
        except:
            # TODO: Alert AI-ML team about a potential change in table/schema
            print('difficulty reading cookie cookie mapping table')
        self.prep_disqus_sizmek_map_table()
        self.prep_zync_sizmek_map_table()
        print('finished step CookieMapUpdate')


In [8]:
config_step_z12 = {
    'path_root': 's3://zeta-dc-ml/ai-audiences/',
    'path_cookie_cookies_relations': 's3://zeta-dcp-prod-private-tables/datacloud_cookie_cookies_relations_links_export/',
}


In [9]:
self = CookieMapUpdate(path_root=config_step_z12['path_root'],
                                         path_cookie_cookies_relations=config_step_z12['path_cookie_cookies_relations'],
                                        )
    